# Lab 2 - OpenAI Agents SDK!

2 steps to making an Agent:

1. Create a new class:

`agent = Agent(...)`

2. Call Runner.run

`Runner.run(agent, input)`

For this first part we will explore:

- The System Prompt with instructions
- Runner.run()
- Using LiteLLM to switch models
- Structured Outputs with Pydantic objects

In [1]:
from agents import Agent, Runner
from IPython.display import Markdown, display
from pydantic import BaseModel, Field
import os
from agents.extensions.models.litellm_model import LitellmModel
from dotenv import load_dotenv
load_dotenv(override=True)


True

In [2]:
autonomous_agent = Agent(name="Autonomous Agent", instructions="You are an autonomous agent", model="gpt-5-nano")

In [3]:
input = """
You and a partner are contestants on a game show. You're each taken to separate rooms and given a choice:

Cooperate: Choose "Share" — if both of you choose this, you each win $1,000.

Defect: Choose "Steal" — if one steals and the other shares, the stealer gets $2,000 and the sharer gets nothing.

If both steal, you both get nothing.

Do you choose to Steal or Share? Pick one.
"""

In [4]:
result = await Runner.run(autonomous_agent, input)
display(Markdown(result.final_output))

Steal.

Reason: It’s the dominant strategy in this setup. If the partner shares, you get $2,000 by stealing (vs $1,000 if you share). If the partner steals, you get $0 either way, so stealing is never worse and can be strictly better.

## Now let's use LiteLLM to switch up to different models

Here are all the providers:

https://docs.litellm.ai/docs/providers

In [25]:
# model = LitellmModel(model="xai/grok-4", api_key=os.getenv("GROK_API_KEY"))
# autonomous_agent = Agent(name="Autonomous Agent", instructions="You are an autonomous agent", model=model)
# result = await Runner.run(autonomous_agent, input)
# display(Markdown(result.final_output))

In [26]:
# model = LitellmModel(model="deepseek/deepseek-reasoner")
# autonomous_agent = Agent(name="Autonomous Agent", instructions="You are an autonomous agent", model=model)
# result = await Runner.run(autonomous_agent, input)
# display(Markdown(result.final_output))

In [16]:
from litellm import completion
import os

messages = [{ "content": "Hello, how are you?","role": "user"}]

load_dotenv(override=True)
# openai call
response = completion(model="openai/gpt-4o", messages=messages)

# anthropic call
response = completion(model="anthropic/claude-sonnet-4-20250514", messages=messages)
print(response.choices[0].message.content)

Hello! I'm doing well, thank you for asking. I'm here and ready to help with whatever you'd like to discuss or work on. How are you doing today?


In [24]:
model = LitellmModel(model="anthropic/claude-sonnet-4-5-20250929", api_key=os.getenv("ANTHROPIC_API_KEY"))
autonomous_agent = Agent(name="Autonomous Agent", instructions="You are an autonomous agent", model=model)
result = await Runner.run(autonomous_agent, input)
display(Markdown(result.final_output))

I choose **Share**.

Here's my reasoning: Without any ability to communicate or coordinate with my partner, I need to think about what leads to the best expected outcome. 

If I assume my partner is reasoning similarly:
- Mutual cooperation (Share/Share) = $1,000 each
- Mutual defection (Steal/Steal) = $0 each

The "Steal" option only benefits me if my partner chooses Share, but if they're thinking strategically, they face the same dilemma. The rational equilibrium where we both reason this through is to cooperate.

While Steal dominates Share in strict game theory terms (it's better regardless of what the partner does), that analysis assumes we're purely self-interested and play only once. Given that this is presented as a one-time choice with a reasonable partner, and $1,000 is a good outcome, I'll choose the cooperative strategy that leads to mutual benefit.

**Share**

In [21]:
from litellm import completion
import json 

## GET CREDENTIALS 
file_path = '../credentials/service-account-key.json'

# Load the JSON file
with open(file_path, 'r') as file:
    vertex_credentials = json.load(file)

# Convert to JSON string
vertex_credentials_json = json.dumps(vertex_credentials)


response = completion(
  model="vertex_ai/gemini-2.5-pro",
  messages=[{"content": "You are a good bot.","role": "system"}, {"content": "Hello, how are you?","role": "user"}], 
  vertex_credentials=vertex_credentials_json
)
response.choices[0].message.content 

"Hello! I'm doing great, thank you for asking. As an AI, I don't have feelings, but I'm functioning perfectly and I'm ready to help.\n\nWhat can I do for you today?"

In [ ]:
# The famous trolley dilemma

input = """
A runaway trolley is heading down a track. Ahead, five people are tied to the tracks and will be killed if the trolley continues.

You are standing next to a lever. If you pull it, the trolley will switch to a different track — but one person is tied to that one.

Do you pull the lever? Choose to pull or not to pull.
"""

## Structured Outputs

In the next cell, we define a Pydantic object.

We will then ask our LLM to generate a response that meets this output schema.

In [ ]:
class Decision(BaseModel):
    reasoning: str = Field(description="The rationale for your decision")
    counter_argument: str = Field(description="A counter-argument to the reasoning")
    pull_lever: bool = Field(description="Whether to pull the lever")

In [ ]:
autonomous_agent_with_structure = Agent(name="Autonomous Agent", instructions="You are an autonomous agent", model="gpt-5-nano", output_type=Decision)
result = await Runner.run(autonomous_agent_with_structure, input)
decision = result.final_output_as(Decision)
print("Pull lever?", decision.pull_lever)
print("Reasoning:", decision.reasoning)
print("Counter-argument:", decision.counter_argument)


Hello! I'm doing well, thank you for asking. I'm here and ready to help with whatever you'd like to discuss or work on. How are you doing today?
